Utiliza o modelo pré treinado (pelo grupo deep learning BSB) da wikipedia português para treinar o classificador de tweets

CLASSIFICAÇÃO BINÁRIA


In [ ]:
import torch
torch.cuda.current_device() # temque chamar essa função primeiro para evitar erro na inicializaçao do CUDA
torch.cuda.set_device(0)

In [ ]:
from fastai import *
from fastai.text import *
import pandas as pd
import numpy as np
import re
import os
path = os.getcwd()
from sklearn import model_selection
print(path)

In [ ]:
print("torch: ", torch.__version__)
print("fastai: ",__version__)

In [ ]:
df = pd.read_pickle('dados\\df_processado.pkl')
df.info()
#df = df[df['sent_manual'].fillna('nan').str.contains('N|E|S|C|D')]
df2 = df[df['sent_manual'].fillna('nan').str.contains('N|E|S|C')].copy()
#df[df['sent_manual'].str.contains('S|D')]['sent_manual'] = 'N'
def corrige_label(label):
    if label == 'S' or label == 'E':
        return('N')
    else:
        return(label)

In [ ]:
df2['sent_manual'] = df2['sent_manual'].apply(corrige_label)
df2['sent_manual'].value_counts()
lista_index = df2.index.values.copy()
lista_texto = df2.tweet_limpo.to_list().copy()
lista_label = df2.sent_manual.to_list().copy()

#TRANSFORMA PARA CAIXA BAIXA
corpus = lista_texto.copy()
for i in range(0,len(corpus)):
    corpus[i]=corpus[i].lower()


#REMOVE NUMEROS E PONTUACAO
for i in range(0,len(corpus)):
    corpus[i] = re.sub('[0-9]+', '', corpus[i])
    corpus[i] = re.sub(r'[^\w\s]','',corpus[i])
    corpus[i] = re.sub('º','',corpus[i])

# create a dataframe using texts and lables
trainDF = pd.DataFrame()
trainDF['text'] = corpus
#trainDF['text'] = lista_texto
trainDF['label'] = lista_label

trainDF['label'].value_counts()

    

In [ ]:
#split dataset
test_size=0.2
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=test_size, random_state = 100)

df_train = pd.DataFrame({
        'texto' : train_x,
        'label' : train_y
        })

df_valid = pd.DataFrame({
        'texto' : valid_x,
        'label' : valid_y
        })

print("df_train.shape =", df_train.shape)
print("df_valid.shape =", df_valid.shape)

Carregando Vocabulário pré-treinado da Wiki em português treinada por integrante do curso NLP Fast.ai

In [ ]:
with open('..\\models\\pt_wt_vocab.pkl', 'rb') as f:
    itos = pickle.load(f)
vocab = Vocab(itos)

In [ ]:
batch_size = 16
data_lm = TextLMDataBunch.from_df(path, 
                                  train_df=df_train, 
                                  valid_df=df_valid,
                                  text_cols=0, 
                                  label_cols=1, 
                                  vocab=vocab, 
                                  min_freq=3, 
                                  bs = batch_size )
len(data_lm.vocab.itos),len(data_lm.train_ds)

In [ ]:
data_lm.vocab.itos[10:20]

Cria o modelo de linguagem em cima do modelo de linguagem pré treinado da wikipedia pt

In [ ]:
learn_lm = language_model_learner(data_lm, 
                                  AWD_LSTM, 
                                  drop_mult=0.3, 
                                  pretrained_fnames=['pt_wt', 'pt_wt_vocab']).to_fp16()

In [ ]:
learn_lm.lr_find()

In [ ]:
learn_lm.recorder.plot(skip_end=15)

In [ ]:
lr = 2e-1
#learn_lm.fit_one_cycle(1, lr, moms=(0.8,0.7))

In [ ]:
learn_lm.save('language_model_fastai_tweets_bc.pkl')

In [ ]:
learn_lm = learn_lm.load('language_model_fastai_tweets_bc.pkl' )

epoch	train_loss	valid_loss	accuracy	time 



epoch	train_loss	valid_loss	accuracy	time <br>
0	10.633375	7.977017	0.147321	00:26 <br>

In [ ]:
#learn_lm.fit_one_cycle(50, lr, moms=(0.8,0.7))
learn_lm.fit_one_cycle(50, lr)

In [ ]:
learn_lm.save('language_model_fastai_tweets_bc.pkl')

In [ ]:
learn_lm.save(f'pt_fine_tuned')
learn_lm.save_encoder(f'pt_fine_tuned_enc')

### Classifier

In [ ]:
data_clas = (TextList.from_df(trainDF, path, vocab=data_lm.vocab, cols='text')
    .split_by_rand_pct(test_size, seed=100)
    .label_from_df(cols='label')
    .databunch(bs=batch_size, num_workers=1))

data_clas.save(f'pt_textlist_class')

In [ ]:
data_clas = load_data(path, f'pt_textlist_class', bs=batch_size, num_workers=1)

In [ ]:
from sklearn.metrics import f1_score, classification_report, cohen_kappa_score

@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))


In [ ]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load_encoder(f'pt_fine_tuned_enc')
learn_c.freeze()

In [ ]:
bs=batch_size
lr=2e-2
lr *= bs/48

In [ ]:
#learn_c.fit_one_cycle(10, lr, moms=(0.8,0.7))
learn_c.fit_one_cycle(10, lr)

In [ ]:
#learn_c.fit_one_cycle(5, lr, moms=(0.8,0.7))
learn_c.fit_one_cycle(5, lr)

In [ ]:
learn_c.freeze_to(-2)
#learn_c.fit_one_cycle(5, slice(lr/(2.6**4),lr), moms=(0.8,0.7))
learn_c.fit_one_cycle(5, slice(lr/(2.6**4),lr))

In [ ]:
learn_c.freeze_to(-3)
#learn_c.fit_one_cycle(5, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))
learn_c.fit_one_cycle(5, slice(lr/2/(2.6**4),lr/2))

In [ ]:
learn_c.unfreeze()
#learn_c.fit_one_cycle(3, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))
learn_c.fit_one_cycle(3, slice(lr/10/(2.6**4),lr/10))

In [ ]:
learn_c.unfreeze()
#learn_c.fit_one_cycle(3, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))
learn_c.fit_one_cycle(3, slice(lr/10/(2.6**4),lr/10))

In [ ]:
learn_c.save(f'pt_clas')

In [ ]:
data_clas = load_data(path, f'pt_textlist_class', bs=bs, num_workers=1)
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load(f'pt_clas', purge=False);

In [ ]:
preds,targs = learn_c.get_preds(ordered=True)
accuracy(preds,targs),f1(preds,targs)

In [ ]:
from fastai.widgets import ClassConfusion
interp = ClassificationInterpretation.from_learner(learn_c)
interp.plot_confusion_matrix()

In [ ]:
print(classification_report(y_true=interp.y_true,
                            y_pred=interp.pred_class))    

In [ ]:
cohen_kappa_score(interp.y_true,
                  interp.pred_class)

In [ ]:
from sklearn.metrics import f1_score, classification_report, cohen_kappa_score, accuracy_score, balanced_accuracy_score, roc_auc_score, recall_score, precision_score
from datetime import datetime

y = interp.y_true
predictions = interp.pred_class

nome_modelo = "ulmfit"

print(classification_report(y_true=y,
                            y_pred=predictions))    

#print("Kappa score: ", cohen_kappa_score(predictions, y),"\n")
#print("Accuracy score: ", accuracy_score(predictions, y),"\n")
#print("f1 macro score: ", f1_score(predictions, y, average='macro'),"\n")
#print("Balanced Accuracy score: ", balanced_accuracy_score(predictions, y))

kappa = cohen_kappa_score(y, predictions)
print("Kappa score: {:.3f}\n".format(kappa))
acc = accuracy_score(y, predictions)
print("Accuracy score: {:.3f}\n".format(acc))
f1 = f1_score(y, predictions, average='weighted')
print("f1 macro score: {:.3f}\n".format(f1))
acc_bal = balanced_accuracy_score(y, predictions)
print("Balanced Accuracy score: {:.3f}\n".format(acc_bal))
roc = roc_auc_score(y, predictions)
print("Area under the ROC curve: {:.3f}\n".format(roc))
rec = recall_score(y, predictions, pos_label = 0, average='binary')
print("Recall classe C: {:.3f}\n".format(rec))
prec = precision_score(y, predictions, pos_label = 0, average='binary')
print("Recall classe C: {:.3f}\n".format(prec))


if nome_modelo:
    dateTimeObj = datetime.now()
    with open("Classificação de tweets\\resultados-classificacao.csv", "a") as myfile:
        myfile.write(nome_modelo+","+str(y.shape[0])+","+str(acc)+","+str(kappa)+","+str(f1)+","+str(acc_bal)+","+str(roc)+","+str(rec)+","+str(prec)+","+dateTimeObj.strftime("%Y-%m-%d")+"\n")